In [1]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox

In [2]:
class App(tk.Tk):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # To do :
        global screen_width
        screen_width = self.winfo_screenwidth()
        global screen_height
        screen_height = self.winfo_screenheight()
        
        global window_width
        window_width = 600
        global window_heigth
        window_height = 900
        
        global center_x 
        center_x = int(screen_width/2 - window_width)
        global center_y 
        center_y = int(screen_height/2 - window_height/3)
        
        self.geometry(f"{window_height}x{window_width}+{center_x}+{center_y}")
        self.title('BiblioMeter Beta')
        
        # Import des bibliothèques nécessaires 
        #import BiblioMeter_GUI as bmg
        import os
        
        # Création de mes deux containers
        container_button = tk.Frame(self, height = 50, bg = 'red')
        container_button.pack(side="top", fill="both", expand=False)
        
        container_frame = tk.Frame(self)
        container_frame.pack(side="top", fill="both", expand=True)
        container_frame.grid_rowconfigure(0, weight=1)
        container_frame.grid_columnconfigure(0, weight=1)
        
        self.frames = {}
        for F in (PageOne, PageFive, PageTwo, PageThree, PageFour):
            page_name = F.__name__
            frame = F(parent = container_frame, controller = self, container_button = container_button)
            self.frames[page_name] = frame

            # put all of the pages in the same location;
            # the one on the top of the stacking order
            # will be the one that is visible.
            frame.grid(row=0, column=0, sticky="nsew")
        
    def show_frame(self, page_name):
        '''Show a frame for the given page name'''
        frame = self.frames[page_name]
        frame.tkraise()

class CheckBoxCorpuses:
    """ Petit automat permettant d'afficher sur la même ligne :
        - L'annee du corpus
        - Wos rawdata/parsing dispo
        - Scopus rawdata/parsing dispo
    """
    
    def __init__(self, parent, year, wos_r, wos_p, scopus_r, scopus_p, concat, *agrs, **kargs):
        self.ESPACE_ENTRE_BOX_CHECK = 100
        self.lab = tk.Label(parent, text = 'Année ' + year)
        
        self.wos_r = tk.Checkbutton(parent)
        if wos_r == True:
            self.wos_r.select()
        self.wos_p = tk.Checkbutton(parent)
        if wos_p == True:
            self.wos_p.select()
        self.scopus_r = tk.Checkbutton(parent)
        if scopus_r == True:
            self.scopus_r.select()
        self.scopus_p = tk.Checkbutton(parent)
        if scopus_p == True:
            self.scopus_p.select()
        self.concat = tk.Checkbutton(parent)
        if concat == True:
            self.concat.select()
    
    def place(self, x, y):
        a = self.lab.winfo_reqwidth()
        self.lab.place(x = x-a, y = y, anchor = 'center')
        self.wos_r.place(x = x+self.ESPACE_ENTRE_BOX_CHECK, y = y, anchor = 'center')
        self.wos_r.config(state = 'disabled')
        self.wos_p.place(x = x+2*self.ESPACE_ENTRE_BOX_CHECK, y = y, anchor = 'center')
        self.wos_p.config(state = 'disabled')
        self.scopus_r.place(x = x+3*self.ESPACE_ENTRE_BOX_CHECK, y = y, anchor = 'center')
        self.scopus_r.config(state = 'disabled')
        self.scopus_p.place(x = x+4*self.ESPACE_ENTRE_BOX_CHECK, y = y, anchor = 'center')
        self.scopus_p.config(state = 'disabled')
        self.concat.place(x = x+5*self.ESPACE_ENTRE_BOX_CHECK, y = y, anchor = 'center')
        self.concat.config(state = 'disabled')
        
    def efface(self):
        for x in (self.lab, self.wos_r, self.wos_p, self.scopus_r, self.scopus_p, self.concat):
            x.place_forget()

class LabelEntry:
    
    """
    Petit automat permettant d'afficher sur la même ligne :
    - un texte d'info
    - une entrée
    - un boutton
    
    Fonctionnalités :
        - l'opt "align" sur la méthode <place> permet d'alligner sur l'entrée plutot que sur le texte.
        - surcharge des méthode get() et set() : pointeur vers le tk.StringVar()
          (permet de garder la continuité des appels sur l'objet Entry créé)
        - permet de masquer/afficher la ligne (<.efface()> / <.place()>) (inutile pour le moment)
        - autorise le replacement (~déplacement) // méthode self.place(x=<float>, y=<float>)
    """

    def __init__(self, parent, text_label, *args, **kargs):
        self.lab = tk.Label(parent, text=text_label)
        self.val = tk.StringVar(parent) # réel associé à la variable "fenetre".
        self.entree = tk.Entry(parent, textvariable=self.val, *args, **kargs)
        self.but = tk.Button(parent, text='...',command = self.get_file)

    def place(self,x,y,align=True):
        a,b = self.lab.winfo_reqwidth(),0
        if not align:
            a,b = b,a
        self.lab.place(x=x-a,y=y)
        self.entree.place(x=x+b,y=y)
        self.but.place(x=x+b+self.entree.winfo_reqwidth()+6,y=y-2)
        
    def get(self):
        return self.val.get()
    
    def set(self, value):
        self.val.set(value)
        
    def get_file(self):
        fic = tk.filedialog.askdirectory(title='Choisir un fichier petit pingouin des Alpes')
        if fic == '':
            return messagebox.showwarning("Attention","Chemin non renseigné")
        self.val.set(fic)
        
    def efface(self):
        for x in (self.lab, self.entree):
            x.place_forget()

####################################################################### PREMIERE PAGE #######################################################################

class PageOne(tk.Frame):    
    def __init__(self, parent, controller, container_button):
        super().__init__(parent)
        self.controller = controller
        
        # Standard library imports
        import os
        
        # 3rd party libray imports
        # None
        
        # Local library imports
        from BiblioMeter_GUI.Globals_GUI import ROOT_PATH
        
        # Liste des checkbox des corpuses
        self.CHECK = list()
        
        #mybutton = tk.Button(self, text = "DONT CLICK ME", command = _show_scopus)
        #mybutton.place(x = 300, y = 350)
            
        label = tk.Label(self, 
                         text="Emplacement de BiblioMeter\n et parsing des fichiers disponibles", 
                         font = ("Helvetica", 25))
        label.place(anchor = 'n', relx = 0.5, rely = 0)
        
        texts = ('Emplacement de BiblioMeter')
        
        # Placement de LABEL ENTRY TO DO
        self.LONGUEUR_ENTRY_FICHIER = 100
        self.POSITION_SELON_X = 200
        self.POSITION_SELON_Y = 70
        self.ESPACE_ENTRE_LIGNE = 30
        
        # Placement de CHECKBOXCORPUSES TO DO
        self.POSITION_SELON_X_CHECK = 245
        self.POSITION_SELON_Y_CHECK = 250
        self.ESPACE_ENTRE_LIGNE_CHECK = 35
        
        self.REP = list()
            
        tmp = LabelEntry(self, text_label=texts+' :', width=self.LONGUEUR_ENTRY_FICHIER)
        tmp.set(ROOT_PATH)
        tmp.place(x=self.POSITION_SELON_X, 
                  y=self.ESPACE_ENTRE_LIGNE+self.POSITION_SELON_Y, 
                  align=True)
        self.REP.append(tmp)
        
        # Faire une vérification de ce qui est présent dans la zone de stockage
        exist_button = tk.Button(self, 
                                 text = 'Vérifier si fichier pour concat sont présents', 
                                 command = lambda: self.call_existing_corpuses(tmp.get()))
        exist_button.place(x = 100, y = 150)
        
        # Mise en page tableau
        wos_rawdata = tk.Label(self, text = 'Wos Rawdata')
        wos_rawdata.place(x = self.POSITION_SELON_X_CHECK + 100, y = 215, anchor = 'center')
        wos_rawdata = tk.Label(self, text = 'Wos Parsing')
        wos_rawdata.place(x = self.POSITION_SELON_X_CHECK + 200, y = 215, anchor = 'center')
        wos_rawdata = tk.Label(self, text = 'Scopus Rawdata')
        wos_rawdata.place(x = self.POSITION_SELON_X_CHECK + 300, y = 215, anchor = 'center')
        wos_rawdata = tk.Label(self, text = 'Scopus Parsing')
        wos_rawdata.place(x = self.POSITION_SELON_X_CHECK + 400, y = 215, anchor = 'center')
        concat = tk.Label(self, text = 'Concatenation \n and dédoublonnage')
        concat.place(x = self.POSITION_SELON_X_CHECK + 500, y = 215, anchor = 'center')
        
        # Mise en place d'un bouton permettant d'avoir accès à PageOne
        button = tk.Button(container_button, 
                           text = 'Choix des chemins', 
                           command = lambda: controller.show_frame("PageOne"))
        button.grid(row = 0, column = 0)
        
    def _open_listbox(self, list_annee):
        import BiblioMeter_GUI as bmg
           
        newWindow = tk.Toplevel(self)
        newWindow.title('Selection des années à concatener')
        
        newWindow.geometry(f"400x400+{self.winfo_rootx()}+{self.winfo_rooty()}")
        
        label = tk.Label(newWindow, text="Selectionner les \n années à concatener", font = ("Helvetica", 20))
        label.place(anchor = 'n', relx = 0.5, rely = 0)
        
        my_listbox = tk.Listbox(newWindow)
        my_listbox.place(anchor = 'center', relx = 0.5, rely = 0.5)
        
        for item in list_annee:
            my_listbox.insert(tk.END, item)
        
        button = tk.Button(newWindow, text ="Lancer la concatenation de l'année sélectionnée", 
                           command = lambda: self.call_concatenate_and_deduplicate(my_listbox.selection_get()))
        button.place(anchor = 'n', relx = 0.5, rely = 0.8)
    
    def call_concatenate_and_deduplicate(self, annee):
        import BiblioMeter_GUI as bmg
        bmg.concatenate_and_deduplicate(annee)
                 
    def call_data_parsing_scopus(self, file, highligth, parsing):
        import BiblioMeter_GUI as bmg
        bmg.data_parsing_scopus(file, highligth, parsing)

    def call_data_parsing_wos(self, file, highligth, parsing):
        import BiblioMeter_GUI as bmg
        bmg.data_parsing_wos(file, highligth, parsing)

    def call_data_parsing(self, year, source, results, second_inst):
        import BiblioMeter_GUI as bmg
        bmg.data_parsing(year, source, results, second_inst)
        
    def call_existing_corpuses(self, root_path):
        
        import BiblioMeter_GUI as bmg
        results = bmg.existing_corpuses(root_path)
        list_annee = results[0]
        list_wos_rawdata = results[1]
        list_wos_parsing = results[2]
        list_scopus_rawdata = results[3]
        list_scopus_parsing = results[4]
        list_concatenation = results[5]
        
        if self.CHECK:
            for x in self.CHECK:
                x.efface()
            for x in self.PARSING:
                x.place_forget()
        
        for i,annee in enumerate(list_annee):
            tmp = CheckBoxCorpuses(self, annee, list_wos_rawdata[i], list_wos_parsing[i], list_scopus_rawdata[i], list_scopus_parsing[i], list_concatenation[i])
            tmp.place(x=self.POSITION_SELON_X_CHECK, y=i*self.ESPACE_ENTRE_LIGNE_CHECK+self.POSITION_SELON_Y_CHECK)
            self.CHECK.append(tmp)
    
        self.PARSING = list()
        label = tk.Label(self, text="Que voulez-vous parser ?", font = ("Helvetica", 14))
        label.place(anchor = 'center', relx = 0.5, y = (len(list_annee)+1)*self.ESPACE_ENTRE_LIGNE_CHECK+self.POSITION_SELON_Y_CHECK-15)
        self.PARSING.append(label)
        
        # CHOIX DE L'ANNEE POUR LE PARSING
        variable_1 = tk.StringVar(self)
        variable_1.set(list_annee[0])
        YearOptionButton = tk.OptionMenu(self, variable_1, *list_annee)
        YearOptionButton.place(anchor = 'center', relx = 0.2, y = (len(list_annee)+2)*self.ESPACE_ENTRE_LIGNE_CHECK+self.POSITION_SELON_Y_CHECK)
        self.PARSING.append(YearOptionButton)
        
        # CHOIX DU TYPE DE DOCUMENT POUR LE PARSING WOS/SCOPUS
        variable_2 = tk.StringVar(self)
        variable_2.set('wos')
        SourceOptionButton = tk.OptionMenu(self, variable_2, *['wos','scopus'])
        SourceOptionButton.place(anchor = 'center', relx = 0.3, y = (len(list_annee)+2)*self.ESPACE_ENTRE_LIGNE_CHECK+self.POSITION_SELON_Y_CHECK)
        self.PARSING.append(SourceOptionButton)
        
        # Effectuer le parsing des secondary institutions ?
        var_second_inst = tk.IntVar()
        check_second_inst = tk.Checkbutton(self, text = "Secondary institutions to be parsed ?", variable = var_second_inst)
        check_second_inst.select()
        check_second_inst.place(anchor = 'center', relx = 0.5, y = (len(list_annee)+2)*self.ESPACE_ENTRE_LIGNE_CHECK+self.POSITION_SELON_Y_CHECK)
        self.PARSING.append(check_second_inst)
        
        # BOUTON POUR LANCER LE PARSING
        launch_button_Wos = tk.Button(self, text = 'Lancement du parsing', command = lambda: self.call_data_parsing(variable_1.get(), 
                                                                                                                    variable_2.get(), 
                                                                                                                    results, 
                                                                                                                    second_inst = var_second_inst.get()))
        launch_button_Wos.place(anchor = 'center', relx = 0.75, y = (len(list_annee)+2)*self.ESPACE_ENTRE_LIGNE_CHECK+self.POSITION_SELON_Y_CHECK)
        self.PARSING.append(launch_button_Wos)
        
        # CONCATENATION DE WOS ET SCOPUS D'UNE ou PLUSIEURS ANNEESl
        label = tk.Button(self, text="Effectuer une concatenation ?", font = ("Helvetica", 14), command = lambda: self._open_listbox(list_annee))
        label.place(anchor = 'center', relx = 0.5, y = (len(list_annee)+4.5)*self.ESPACE_ENTRE_LIGNE_CHECK+self.POSITION_SELON_Y_CHECK-15)
        self.PARSING.append(label)
        

####################################################################### DEUXIEME PAGE #######################################################################
class PageTwo(tk.Frame):    
    def __init__(self, parent, controller, container_button):
        super().__init__(parent)
        self.controller = controller

        # Standard library imports
        import os
        
        # 3rd party libray imports
        # None
        
        # Local library imports
        from BiblioMeter_GUI.Globals_GUI import ROOT_PATH
        
        label = tk.Label(self, 
                         text="Analyse multi mensuelle", 
                         font = ("Helvetica", 25))
        label.place(anchor = 'n', relx = 0.5, rely = 0)
        
        #my_image = ImageTk.PhotoImage(self, Image.open(r"C:\Users\LD259969\Pictures\GraphEvolution.png"))
        #label_image = tk.Label(image = my_image)
        
        #label_image.place(anchor = 'center', relx = 0.5, rely = 0.5)
        
        # Récupérer les corpus disponibles
        list_dir = os.listdir(ROOT_PATH)
        list_annee = list()
        for annee in list_dir:
            if len(annee) == 4:
                list_annee.append(annee)
                
        variable = tk.StringVar(self)
        variable.set(list_annee[0])

        OptionButton = tk.OptionMenu(self, variable, *list_annee)
        OptionButton.place(anchor = 'n', relx = 0.2, rely = 0.1)


        button = tk.Button(container_button, text = 'Analyse multi mensuelle', command = lambda: controller.show_frame("PageTwo"))
        button.grid(row = 0, column = 2)


####################################################################### TROISIEME PAGE #######################################################################
class PageThree(tk.Frame):    
    def __init__(self, parent, controller, container_button):
        super().__init__(parent)
        self.controller = controller
        
        label = tk.Label(self, 
                         text="Analyse multi annuelle", 
                         font = ("Helvetica", 25))
        label.place(anchor = 'n', relx = 0.5, rely = 0)
        
        button = tk.Button(container_button, text = 'Analyse multi annuelle', command = lambda: controller.show_frame("PageThree"))
        button.grid(row = 0, column = 3)

####################################################################### QUATRIEME PAGE #######################################################################
class PageFour(tk.Frame):    
    def __init__(self, parent, controller, container_button):
        super().__init__(parent)
        self.controller = controller
        
        label = tk.Label(self, 
                         text="Filtres à OTP", 
                         font = ("Helvetica", 25))
        label.place(anchor = 'n', relx = 0.5, rely = 0)
        
        Button_DTNM = tk.Button(self, text = 'DTNM', 
                                font = ("Helvetica", 18))
        Button_DTNM.place(anchor = 'center', relx = 0.25, rely = 0.25)
        
        Button_DTBH = tk.Button(self, text = 'DTBM', 
                                font = ("Helvetica", 18))
        Button_DTBH.place(anchor = 'center', relx = 0.75, rely = 0.25)
        
        Button_DTS = tk.Button(self, text = 'DTS', 
                               font = ("Helvetica", 18))
        Button_DTS.place(anchor = 'center', relx = 0.25, rely = 0.75)
        
        Button_DEHT = tk.Button(self, text = 'DEHT', 
                                font = ("Helvetica", 18))
        Button_DEHT.place(anchor = 'center', relx = 0.75, rely = 0.75)
        
        button = tk.Button(container_button, text = 'OTP', command = lambda: controller.show_frame("PageFour"))
        button.grid(row = 0, column = 4)

####################################################################### CINQUIEME PAGE #######################################################################
class PageFive(tk.Frame):    
    def __init__(self, parent, controller, container_button):
        super().__init__(parent)
        self.controller = controller
        
        # Standard library imports
        import os
        from pathlib import Path
        
        # 3rd party libray imports
        # None
        
        # Local library imports
        from BiblioMeter_GUI.Globals_GUI import ROOT_PATH
        
        label = tk.Label(self, 
                         text="Création des fichiers effectifs RH", 
                         font = ("Helvetica", 25))
        label.place(anchor = 'n', relx = 0.5, rely = 0)
        
        # Récupérer les corpus disponibles
        list_dir = os.listdir(ROOT_PATH)
        list_annee = list()
        for annee in list_dir:
            if len(annee) == 4:
                list_annee.append(annee)
                
        variable = tk.StringVar(self)
        variable.set(list_annee[0])

        OptionButton = tk.OptionMenu(self, variable, *list_annee)
        OptionButton.place(anchor = 'center', relx = 0.20, rely = 0.20)
        
        Button_merged = tk.Button(self, text = "Création de la liste des auteurs Liten à \n partir des fichiers concaténés et dédoublonnés \n pour l'année sélectionnée", 
                                  font = ("Helvetica", 9), 
                                  command = lambda: self.call_build_pubs_authors_Liten(variable.get()))
        Button_merged.place(anchor = 'center', relx = 0.45, rely = 0.20)
        
        Button_single_year = tk.Button(self, text = 'PubliXEffectifs Single Year', 
                                  font = ("Helvetica", 18), 
                                  command = lambda: self.call_single_year_search(variable.get()))
        Button_single_year.place(anchor = 'center', relx = 0.45, rely = 0.50)
        
        Button_recursive = tk.Button(self, text = 'PubliXEffectifs Recursive', 
                                  font = ("Helvetica", 18), 
                                  command = lambda: self.call_recursive_year_search(variable.get()))
        Button_recursive.place(anchor = 'center', relx = 0.45, rely = 0.70)
        
        button = tk.Button(container_button, text = 'Documents RH', command = lambda: controller.show_frame("PageFive"))
        button.grid(row = 0, column = 1)
        
    def call_build_pubs_authors_Liten(self, annee):
        
        # Standart library imports
        from pathlib import Path
        
        # Local library imports
        from BiblioMeter_GUI.Globals_GUI import ROOT_PATH
        import BiblioMeter_GUI as bmg
        
        df_pub = bmg.build_pubs_authors_Liten(annee)
        df_pub.to_excel(Path(ROOT_PATH) / Path(annee) / Path('Auteurs Liten Publiants') / Path(f'Authors_Liten_{annee}.xlsx'))
        print('The articles dataframe is built and saved as EXCEL file')
        
    def call_single_year_search(self, annee):
        
        # Standart library imports
        from pathlib import Path
        
        # Local library imports
        from BiblioMeter_GUI.Globals_GUI import EFFECTIF_PATH
        from BiblioMeter_GUI.Globals_GUI import ROOT_PATH
        import BiblioMeter_GUI as bmg
        bmg.single_year_search(annee)
        
    def call_recursive_year_search(self, annee):
        
        # Standart library imports
        from pathlib import Path
        
        # Local library imports
        from BiblioMeter_GUI.Globals_GUI import EFFECTIF_PATH
        from BiblioMeter_GUI.Globals_GUI import ROOT_PATH
        import BiblioMeter_GUI as bmg
        bmg.recursive_year_search(annee)

In [3]:
app = App()
app.mainloop()

C:\Users\ld259969\Documents\PyVenv\BiblioMeterDraft\venv\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Results saved
Results saved
Results saved

Similarities by orphan reduction for author lastname: TUAN TRAN
  Lastname flag match: True
  Nb similarities by orphan reduction: 1
  List of lastnames with similarities: ['TRAN']
  Employee matricules: [231490, 256756, 267273]
  Employee lastnames: ['TRAN', 'TRAN', 'TRAN']
  Employee firstnames: ['QUOC-TUAN', 'THI-BICH HUE', 'NGOC-ANH']
  Employee fullnames: ['TUAN TRAN QT', 'TUAN TRAN TBH', 'TUAN TRAN NA']

Initiales for author lastname: TUAN TRAN
  Author fullname: TUAN TRAN Q
  Author firstname initiales: Q

Initiales of matching employees for author lastname: TUAN TRAN
  Employees firstname initiales list: ['QT', 'TBH', 'NA']

Checking initiales matching for author lastname: TUAN TRAN
  Nb of matching initiales: 1
  Index list of matching initiales: [0]
  Employees lastnames list: ['TRAN', 'TRAN', 'TRAN']
Results saved

Similarities by orphan reduction for author lastname: TUAN TRAN
  Lastname flag match: True
  Nb similarities by orphan

In [ ]:
from tkinter import *
from tkinter import ttk

ws = Tk()
ws.title('PythonGuides')
ws.geometry('400x300')
ws['bg']='#fb0'

tv = ttk.Treeview(ws)
tv['columns']=('Rank', 'Name', 'Badge')
tv.column('#0', width=0, stretch=NO)
tv.column('Rank', anchor=CENTER, width=100)
tv.column('Name', anchor=CENTER, width=80)
tv.column('Badge', anchor=CENTER, width=80)

tv.heading('#0', text='', anchor=CENTER)
tv.heading('Rank', text='Id', anchor=CENTER)
tv.heading('Name', text='rank', anchor=CENTER)
tv.heading('Badge', text='Badge', anchor=CENTER)

tv.insert(parent='', index=0, iid=0, text='', values=('1','Vineet','Alpha'))
tv.insert(parent='', index=1, iid=1, text='', values=('2','Anil','Bravo'))
tv.insert(parent='', index=2, iid=2, text='', values=('3','Vinod','Charlie'))
tv.insert(parent='', index=3, iid=3, text='', values=('4','Vimal','Delta'))
tv.insert(parent='', index=4, iid=4, text='', values=('5','Manjeet','Echo'))
tv.pack()


ws.mainloop()

Voir pour les imports, faire attention à bien séparer les actions (ex: GUI/Parsing)
Et voir super().__init()check_button_ConcatYearOptionButton